In [ ]:
# | default_exp solvemodels

<span style="color:red">
    <font size="6">
Remember to change maintenance ratio in 2.1 and supermodel.py
    </font>
</span>

## Setup

In [ ]:
# | export


import json
import sys

import cobra

# from x import y syntax doesn't work because of nbdev export format
import mmon_gcm.buildingediting
import mmon_gcm.solving
import mmon_gcm.supermodel
import pandas as pd

In [ ]:
# This cell isn't exported to the .py file, so define here if running in notebook rather than as .py on e.g.a cluster
# This is where to adjust whether fva is run if running in notebook
args = {
    "run_fva": "True",
    "no_processes": 6,
    "output_dir": "../outputs/model_solutions/",
    "model_file": "../models/4_stage_GC.xml",
    "map_file": "../inputs/map.json",
    "parameters_file": "../inputs/arabidopsis_parameters.csv",
}

sys.argv = ["script_name"] + list(args.values())

In [ ]:
# | export

run_fva = sys.argv[1]
if run_fva == "True":
    run_fva = True
elif run_fva == "False":
    run_fva = False
else:
    raise ValueError(f"Please specify True or False for run_fva, not {run_fva}")

no_processes = int(sys.argv[2])
output_dir = sys.argv[3]
model_file = sys.argv[4]
map_file = sys.argv[5]
parameters_file = sys.argv[6]

## Instantiate and apply base constraints to model

### Import FBA model

In [ ]:
# | export

four_stage_GC_model = cobra.io.sbml.read_sbml_model(model_file)  # read model

No objective coefficients in model. Unclear what should be optimized


Decrease tolerance

In [ ]:
# | export

print(four_stage_GC_model.solver.configuration.tolerances.integrality)
print(four_stage_GC_model.solver.configuration.tolerances.feasibility)
four_stage_GC_model.solver.configuration.tolerances.feasibility = 1e-7
print(four_stage_GC_model.solver.configuration.tolerances.feasibility)

1e-07
1e-07
1e-07


### Define reactions to use for FVA

In [ ]:
# | export

if run_fva == True:
    linker_list = [
        reaction
        for reaction in four_stage_GC_model.reactions
        if "gc_Linker_2" in reaction.id
        or "gc_Linker_1" in reaction.id
        or "_total_pseudolinker_1" in reaction.id
        or "_total_pseudolinker_2" in reaction.id
        or "ae_gc" in reaction.id
    ]

    extra_fva_rxns = set()

    with open(map_file, "r+") as f:
        map_data = json.load(f)

    for reaction in map_data[1]["reactions"].values():
        extra_fva_rxns.add(four_stage_GC_model.reactions.get_by_id(reaction["bigg_id"]))

    fva_list = list(set(linker_list) | extra_fva_rxns)

else:
    fva_list = []

### Instantiate Supermodel

Import parameters from csv, created in [parameters notebook](1.2_defining_parameters.ipynb)

In [ ]:
# | export

parameters_df = pd.read_csv(parameters_file, index_col=0)
parameters_df

,Value,Units,Source
P_abs,9.000000e-01,Dimensionless,"Zhu, Long, and Ort (2010)"
T_l,1.700000e-04,m,Wuyts et al. (2010)
A_l,1.000000e+00,m$^2$,Fixed
V_gc_ind,4.750000e-13,dm$^3$,Jezek and Blatt (2017)
FqFm,9.000000e-01,Dimensionless,Lawson (2003)
R_ch,6.923077e-02,Dimensionless,"Fujiwara, Sanjaya, and Itoh (2019)"
R_ch_vol,2.004760e-01,Dimensionless,Knoblauch et al. (2023)
L_air,3.700000e-01,Dimensionless,Earles et al. (2018)
L_epidermis,1.500000e-01,Dimensionless,Wuyts et al. (2010)
Vac_frac,7.510000e-01,Dimensionless,Wang et al. (2017)


In [ ]:
# | export

arabidopsis_supermodel = mmon_gcm.supermodel.SuperModel(parameters_df.loc[:, "Value"], fba_model=four_stage_GC_model)

In [ ]:
# | export

arabidopsis_supermodel.get_volumes(printouts=True);

Volume in phase 0 is 0.38pL
Volume in phase 1 is 0.4375pL
Volume in phase 2 is 0.38pL
Volume in phase 3 is 0.38pL


In [ ]:
# | export

arabidopsis_supermodel.get_volumes(printouts=True, per_guard_cell=False);

Volume in phase 0 is 0.0002204dm3
Volume in phase 1 is 0.00025375dm3
Volume in phase 2 is 0.0002204dm3
Volume in phase 3 is 0.0002204dm3


This is a little different to Jezek and Blatt (2017), where they have closed as 0.3-0.4pL and open as 0.5-0.65. Open apertures are on the small side but still within their range (2-6). Wang used a volume of 0.4pL.

### Constrain SuperModel

#### Constrain osmolarity of the model using the equation from the OnGuard model (Hills et al 2012)

In [ ]:
# | export

arabidopsis_supermodel.constrain_osmolarity(printouts=True);

Raw osmolarities: [0.039359327616195004, 0.05492236409486689, 0.039359327616195004, 0.039359327616195004]
Osmolarities in mM: [178.58134127 216.44281417 178.58134127 178.58134127]
Change in osmolarity: 37.86147289566091mM
c osmolarities constrained to [0.00980047 0.01367567 0.00980047 0.00980047]
v osmolarities constrained to [0.02955886 0.0412467  0.02955886 0.02955886]


In [ ]:
import numpy as np

In [ ]:
np.array(arabidopsis_supermodel.get_osmolarities())

array([0.03935933, 0.05492236, 0.03935933, 0.03935933])

In [ ]:
help(arabidopsis_supermodel.get_osmolarities)

Help on method get_osmolarities in module mmon_gcm.supermodel:

get_osmolarities(apertures='default', equation='onguard', printouts=False) method of mmon_gcm.supermodel.SuperModel instance
    Calculates the osmolarities at each phase of the model using the equation
    specified, default is onguard. Adds these as an attribute to the SuperModel



In [ ]:
mmoles_per_m2 = np.array(arabidopsis_supermodel.get_osmolarities())
mmoles_per_gc = mmoles_per_m2 / arabidopsis_supermodel.N_gcs
moles_per_gc = mmoles_per_gc * 10**-3
fmoles_per_gc = moles_per_gc / 10**-15
print(f"Closed fmoles: {fmoles_per_gc[0]}")
print(f"Open fmoles: {fmoles_per_gc[1]}")
print(f"Increase in fmoles: {fmoles_per_gc[1] - fmoles_per_gc[0]}")

Closed fmoles: 67.86090968309483
Open fmoles: 94.69373119804636
Increase in fmoles: 26.832821514951533


#### Constrain photons using a PPFD of 150µmolm$^{-2}$s$^{-1}$, same as used in Horrer et al (2016)

In [ ]:
# | export

PPFD = 150
arabidopsis_supermodel.constrain_photons(PPFD, printouts=True);

Total leaf volume: 0.17dm3
Guard cell volume in 1m2 leaf: 0.0003dm3
Mesophyll cell volume in 1m2 leaf: 0.091dm3
Proportion of the leaf that is gc: 0.003
PPFD 150umolphotonsm-2
Photon influx into leaf: 486.0 mmolphotonsm-2hr-1
e = 0.012
Photon influx into Guard cells: 0.018mmolphotonsm-2hr-1
Photon influx into mesophyll cells: 485.982mmolphotonsm-2hr-1


## Run FBA without maintenance to get sum of fluxes

In [ ]:
no_maintenance_solutions = pd.DataFrame()

### Blue Light Unconstrained H+-ATPase WT

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound) 
    (
        blue_unconstrained_wt,
        blue_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['blue_uncon_wt'] = blue_unconstrained_wt_solution['fluxes']

### Blue Light Unconstrained H+-ATPase Starch KO

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound) 
    (
        blue_unconstrained_ko,
        blue_unconstrained_ko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['blue_uncon_ko'] = blue_unconstrained_ko_solution['fluxes']

### Blue Light Constrained H+-ATPase WT

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)
    (
        blue_constrained_wt,
        blue_constrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['blue_con_wt'] = blue_constrained_wt_solution['fluxes']

### Blue Light Constrained H+-ATPase Starch KO

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound) 
    (
        blue_constrained_ko,
        blue_constrained_ko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['blue_con_ko'] = blue_constrained_ko_solution['fluxes']

### White Light Unconstrained H+-ATPase WT

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound) 
    (
        white_unconstrained_wt,
        white_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['white_uncon_wt'] = white_unconstrained_wt_solution['fluxes']

### White Light Unconstrained H+-ATPase Starch KO

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound) 
    (
        white_unconstrained_ko,
        white_unconstrained_ko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['white_uncon_ko'] = white_unconstrained_ko_solution['fluxes']

### White Light Constrained H+-ATPase WT

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)
    (
        white_constrained_wt,
        white_constrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['white_con_wt'] = white_constrained_wt_solution['fluxes']

### White Light Constrained H+-ATPase Starch KO

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound) 
    (
        white_constrained_ko,
        white_constrained_ko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['white_con_ko'] = white_constrained_ko_solution['fluxes']

### No PS Unconstrained H+-ATPase WT

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound) 
    (
        nops_unconstrained_wt,
        nops_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['nops_uncon_wt'] = nops_unconstrained_wt_solution['fluxes']

### No PS Unconstrained H+-ATPase Starch KO

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound) 
    (
        nops_unconstrained_ko,
        nops_unconstrained_ko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['nops_uncon_ko'] = nops_unconstrained_ko_solution['fluxes']

### No PS Constrained H+-ATPase WT

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)
    (
        nops_constrained_wt,
        nops_constrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['nops_con_wt'] = nops_constrained_wt_solution['fluxes']

### No PS Constrained H+-ATPase Starch KO

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound) 
    (
        nops_constrained_ko,
        nops_constrained_ko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['nops_con_ko'] = nops_constrained_ko_solution['fluxes']

Started running pFBA (and FVA) @ 2024-05-29 11:00:18.771459
Running pFBA
FVA list is empty
Finished running pFBA (and FVA) @ 2024-05-29 11:00:32.121406, that took 0.2224991202354431 minutes
Started running pFBA (and FVA) @ 2024-05-29 11:00:32.142613
Running pFBA
FVA list is empty
Finished running pFBA (and FVA) @ 2024-05-29 11:00:45.305443, that took 0.21938050587972005 minutes
Started running pFBA (and FVA) @ 2024-05-29 11:00:45.317288
Running pFBA
FVA list is empty
Finished running pFBA (and FVA) @ 2024-05-29 11:01:01.051978, that took 0.26224482456843057 minutes
Started running pFBA (and FVA) @ 2024-05-29 11:01:01.061748
Running pFBA
FVA list is empty
Finished running pFBA (and FVA) @ 2024-05-29 11:01:16.650575, that took 0.25981378157933555 minutes
Started running pFBA (and FVA) @ 2024-05-29 11:01:16.676700
Running pFBA
FVA list is empty
Finished running pFBA (and FVA) @ 2024-05-29 11:01:31.049585, that took 0.23954807917277018 minutes
Started running pFBA (and FVA) @ 2024-05-29 11

### Get the list of reactions to exclude from sum of fluxes calculation

In [ ]:
sum_fluxes_exclude = set()
for r in list(no_maintenance_solutions.index):
    parts = r.split('_')
    if parts[-1] in ['gc', 'me', 'day', 'overall', 'night']: # not in ['1', '2', '3', '4']
        sum_fluxes_exclude.add(r)
    else:
        if parts[-2] in ['tx', 'pseudolinker', 'Linker', 'a']: # not in ['me', 'gc']
            sum_fluxes_exclude.add(r)
        else:
            if parts[-3] in ['ae', 't', 'b', 'ep', 'tx', 'biomass']: # not in ['cv', 'p', 'xc', 'pc', 'mc', 'im', 'vc', 'mi', 'v', 'c', 'm', 'x', 'ec', 'pr', 'r', 'ce', 'cwINV']
                sum_fluxes_exclude.add(r)
            else:
                if 'constraint' in r:
                    sum_fluxes_exclude.add(r)

In [ ]:
# Filter solution with only metabolic reactions and transporters (with constrained H+-ATPase only)
filtered_df = no_maintenance_solutions.loc[~no_maintenance_solutions.index.isin(sum_fluxes_exclude), no_maintenance_solutions.columns.str.contains('_con_')]

In [ ]:
phase_length = {'1': 6, '2': 0.5, '3': 11.5, '4': 6}

def multiply_by_hours(row):
    index_suffix = row.name.split('_')[-1]
    factor = phase_length[index_suffix]
    return row * factor

scaled_df = filtered_df.apply(multiply_by_hours, axis=1)

#gc_sum_flux = filtered_df[filtered_df.index.str.contains('|'.join(['_gc_1', '_gc_4']))].abs().sum()
#me_sum_flux = filtered_df[filtered_df.index.str.contains('|'.join(['_me_1', '_me_4']))].abs().sum()
gc_sum_flux = scaled_df[scaled_df.index.str.contains('_gc_')].abs().sum()
me_sum_flux = scaled_df[scaled_df.index.str.contains('_me_')].abs().sum()

In [ ]:
#print(scaled_df)
print(gc_sum_flux)
print(me_sum_flux)

blue_con_wt     0.901933
blue_con_ko     0.899520
white_con_wt    0.918848
white_con_ko    0.921276
nops_con_wt     0.201886
nops_con_ko     0.214510
dtype: float64
blue_con_wt     20142.122686
blue_con_ko     20142.122755
white_con_wt    20994.863369
white_con_ko    20994.863952
nops_con_wt     20994.859108
nops_con_ko     20994.859400
dtype: float64


In [ ]:
gc_me_ratio = gc_sum_flux/me_sum_flux
print(gc_me_ratio)

blue_con_wt     0.000045
blue_con_ko     0.000045
white_con_wt    0.000044
white_con_ko    0.000044
nops_con_wt     0.000010
nops_con_ko     0.000010
dtype: float64


In [ ]:
gc_me_ratio.head(4).mean()

4.427086760474383e-05

Ratio of GC:ME maintenance = 4.427086760474383e-05
(Update value in 2.1_solvingmodel AND in supermodel.py)